# Python 的基礎 - Package 的使用與安裝 2

剛剛我們成功連線到 Trinity Repository 並能夠拉取裡面的資料了

現在可以搭配剛才的輸出檔案，將想要的資料輸出成報表

不多說，小練習 GO

## 小練習 1

請盤點出剛才的 Job 的 Script，並輸出成報表

`````{admonition} For Your Information
剛才的 Job 是在 「 康是美 」 使用的 Tableau Command refreshextracts

JobStep 全部都是 External Command ， 並且是以 EMBEDDED_SCRIPT 執行

這邊我們就不解釋怎麼從 stepxml 中 ， 取出 Command ， SQL 如下

````{admonition} SQL
```sql
select b2.busentityname, j2.categoryname, j.jobname, js.stepname
     , unnest(xpath('//prop[@name="command"]/@value', js.xmldata::xml))::text as COMMAND
     , unnest(xpath('//prop[@name="scriptsql"]/@value', js.xmldata::xml))::text as EMBEDDED_SCRIPT
from job j
left join jobstep js on j.jobuid = js.jobuid
left join jobcategory j2 on j.categoryuid = j2.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where j.activate = '1'
  and b2.busentityname = 'Z_COSMED'
  and j2.categoryname = 'EXT'

```

````

`````

### 小練習 1 參考解答

In [9]:
import psycopg2

# 連線 94 這麼簡單
connection = psycopg2.connect(host='10.140.0.2', dbname='trinity', user='trinity', password='trinity')

# Create Cursor
cursor = connection.cursor()

# 三個單引號 --> String 
sql = """
select b2.busentityname, j2.categoryname, j.jobname, js.stepname
     , unnest(xpath('//prop[@name="command"]/@value', js.xmldata::xml))::text as COMMAND
     , unnest(xpath('//prop[@name="scriptsql"]/@value', js.xmldata::xml))::text as EMBEDDED_SCRIPT
from job j
left join jobstep js on j.jobuid = js.jobuid
left join jobcategory j2 on j.categoryuid = j2.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where j.activate = '1'
  and b2.busentityname = 'Z_COSMED'
  and j2.categoryname = 'EXT'
"""

# 執行 SQL
cursor.execute(sql)

f = open('file/report.csv', 'w', encoding='UTF-8')

# Set Columns
column = ('busentityname','categoryname','jobname','stepname','command','embedded_script',)
f.write(','.join(column))
f.write('\n')

# cursor object is iterable
for rs in cursor:
    f.write(','.join(rs))
    f.write('\n')

f.close()

輸出成報表後

用 Excel 給他打開下去吧！

```{note}
這次輸出的報表中沒有中文

可以放心直接用 excel 打開
```

## 套件 Pandas

------

如果你用 `Notepad++` 之類的編輯器打開剛才的報表檔案 CSV 

會發現 Excel 中 Embedded_script 裡 `"${TABLEAU_PATH}"` 的雙引號不見了

這是因為我們沒有特別去處理 <b> `雙引號` </b> 的問題

CSV 標準格式中有如下的定義

```{important}
僅節錄部分

<b><font size=4> RFC 4180 standard </font></b>

- Any field may be quoted (with double quotes).

- Fields containing a line-break, double-quote or commas should be quoted. (If they are not, the file will likely be impossible to process correctly.)

- If double-quotes are used to enclose fields, then a double-quote in a field must be represented by two double-quote characters.

```

上述標準簡單來說

- 一個欄位的值，若包含 <b><font size=2> `換行 、  雙引號 、 逗號` </font></b> 的話，必須以<b><font size=2> `雙引號` </font></b>來包覆

- 如果要使用包含<b><font size=2> `雙引號` </font></b>的欄位值，則必須以兩個<b><font size=2> `雙引號` </font></b>表示

那我們勢必在寫檔時，要去替換有雙引號的地方

很麻煩吧 ！ 

<b><font size=4> 沒事 ！ </font></b> 這邊有好用的套件 <b><font size=3> `Pandas` </font></b> 幫我們處理

### 安裝 Pandas

一樣使用 `pip` 來安裝

如果你在安裝 Python 時選用 `Anaconda` 的安裝包的話，不必再裝，`Anaconda` 有自帶

```
pip3 install pandas
```

測試是否安裝成功

```
python3 -c 'import pandas as pd'
```


````{note}
前面我們 import 套件後

都是這樣用 

```
import psycopg2


psycopg2.FUNCTION_NAME
```

這邊我自己的習慣會簡化 `pandas` 的名子

```
import pandas as pd


pd.FUNCTION_NAME
```
````

## 使用 Pandas

Pandas 是一個高效能且強力的資料分析套件

它提供了 DataFrame 的資料型態 ， 非常像 Excel 的資料表

而且可以讀取 Excel(`.xlsx .xls`) 、 CSV(`.csv`) 還可以直接讀取 DB Table

這邊我們來直接讀 DB Table ， 並且輸出成 CSV

### Pandas Read DB

`pandas`.<b><font size=3>`read_sql`</font></b>(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)


重要的大概是前兩個 `sql` & `con`

其他的詳細請看 [Doc](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)

- <b><font size=3>`sql`</font></b>

可以直接用 String 的方式傳入 SQL Statement

- <b><font size=3>`con`</font></b>

這個稍微特殊一點

目前還可以使用 DBAPI2 Object

也就是 `psycopg2` 的 Connection Object 可以使用

```{note}
Pandas 文件中寫道：

<b>con : SQLAlchemy connectable, str, or sqlite3 connection</b>

Using SQLAlchemy makes it possible to use any DB supported by that library. <b>If a DBAPI2 object, only sqlite3 is supported. </b> The user is responsible for engine disposal and connection closure for the SQLAlchemy connectable. 

目前還可以用，但未來不確定是否能繼續用 `psycopg2` 的 Connection

文件中建議使用 `SQLAlchemy` Library

[只稍微看一下](https://docs.sqlalchemy.org/en/14/dialects/postgresql.html#module-sqlalchemy.dialects.postgresql.psycopg2)

```

In [10]:
import pandas as pd

df = pd.read_sql(sql=sql, con=connection)

df

,busentityname,categoryname,jobname,stepname,command,embedded_script
0,Z_COSMED,EXT,EXT_FINTARGETTEAM,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
1,Z_COSMED,EXT,EXT_STOCKSALES_DATETYPE,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
2,Z_COSMED,EXT,EXT_STOREHOURSALES,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
3,Z_COSMED,EXT,EXT_TAXREFUND,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
4,Z_COSMED,EXT,EXT_TANAITEMSALES,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
5,Z_COSMED,EXT,EXT_TANADETAILSALES_MON,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
6,Z_COSMED,EXT,EXT_TANADETAILSALES,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
7,Z_COSMED,EXT,EXT_STORESALESTARGET_MONTH,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
8,Z_COSMED,EXT,EXT_STORESALESTARGET,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."
9,Z_COSMED,EXT,EXT_STOREITEMSALES,EXTRACT,${EMBEDDED_SCRIPT},"""${TABLEAU_PATH}"" refreshextracts --server ${T..."


### Pandas Export CSV

最後，讓 Pandas 幫我們輸出 CSV

In [11]:
df.to_csv('file/report_pandas.csv', index=False)

```{note}
這邊的 `index` 參數，是指是否要輸出在 `Pandas` 中使用的 `index`

通常我都不輸出，畢竟那個 `index` 只有 `Pandas` 看得懂
```

可以比較看看 `report.csv` 與 `report_pandas.csv` 的差別

可以發現 `Pandas` 幫我們完成了雙引號的處理

一個欄位值用雙引號包覆

使用到雙引號的話，就用兩個雙引號表示

![img](static/pandascsv.png)

再用 Excel 打開看看， 應該會正確處理雙引號了

![img](static/excelcsv.png)